## Titanic Kaggle Project: Support Vector Machine Model

In this notebook, I build a Support Vector Machine classifier to predict survivors of the Titanic. 

In [51]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import pickle as pkl

In [52]:
# Load data
df = pd.read_csv('../../Data/Titanic/train.csv')
submission_data = pd.read_csv('../../Data/Titanic/test.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [54]:
# Functions for feature engineering 
def get_titles(data):
    titles = []
    for name in data['Name']:
        title = name.split(',')[1].split('.')[0].strip()
        if title in ['Ms', 'Miss', 'Mrs', 'Mme', 'Lady', 'Mr', 'Dr', 'Rev', 'Master', 'Major', 'Col', 'Capt']:
            titles.append(title)
        else:
            titles.append('Other')
    return titles

def get_cabin(data):
    cabins = []
    for cabin in data['Cabin']:
        if pd.isna(cabin):
            cabins.append('U')
        else:
            cabins.append(cabin[0])
    return cabins

# Apply initial preprocessing on name and cabin
def preprocessing(data, is_submission=False):
    '''
    This function takes in the dataframe and applies feature engineering on the Name and Cabin features.
    It returns a new dataframe that is ready for further preprocessing in the pipeline.

    Parameters:

    Input:
        data: The Pandas dataframe to be processed
        is_submission: Boolean to indicate if the data is the submission data (the data that will be submitted to Kaggle) or not 

    Returns:
        data: The Pandas dataframe with feature engineering applied on the Name and Cabin features
    '''
    if is_submission:
        data = submission_data.copy()
        data.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)
        data['Title'] = get_titles(submission_data)
        data.drop('Name', axis=1, inplace=True)
        data['Cabin'] = get_cabin(submission_data)
        return data, submission_data['PassengerId']
    else:
        df.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)
        data['Title'] = get_titles(data)
        data.drop('Name', axis=1, inplace=True)
        data['Cabin'] = get_cabin(data)
    return data
    
df = preprocessing(df)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,U,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,C,Mrs
2,1,3,female,26.0,0,0,7.9250,U,S,Miss
3,1,1,female,35.0,1,0,53.1000,C,S,Mrs
4,0,3,male,35.0,0,0,8.0500,U,S,Mr
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,U,S,Rev
887,1,1,female,19.0,0,0,30.0000,B,S,Miss
888,0,3,female,NaN,1,2,23.4500,U,S,Miss
889,1,1,male,26.0,0,0,30.0000,C,C,Mr


In [58]:
# Split data into Train and Test
X_train, X_test, y_train, y_test = train_test_split(df.drop('Survived', axis=1), df['Survived'], test_size=0.2, random_state=42)

In [59]:
# Separate features and create pipelines
num_featuress = ['Age', 'SibSp', 'Parch', 'Fare']
cat_simple_features = ['Sex', 'Cabin', 'Embarked', 'Title'] 
ord_features = ['Pclass']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('sqrt', FunctionTransformer(np.sqrt)),
    ('std_scaler', StandardScaler())
])
cat_simple_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder())
]) 
ord_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ord', OrdinalEncoder())
]) 

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_featuress),
    ('cat_simple', cat_simple_pipeline, cat_simple_features),
    ('ord', ord_pipeline, ord_features)
])

In [60]:
# Fit and transform data
X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)

In [61]:
# Apply grid search to find best parameters
grid = {
    'C': [0.01, .1, 1, 10, 100],
    'gamma': [.001, .01, .1, 1],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

grid_search = GridSearchCV(SVC(), grid, verbose=2, cv=5, n_jobs=-1)
grid_search.fit(X_train_prepared, y_train)
grid_search.best_score_

Fitting 5 folds for each of 80 candidates, totalling 400 fits


0.8342460356544862

In [62]:
# Obtain best model
model = grid_search.best_estimator_

In [66]:
# Gather predictions on submission data
submission_df, submission_passengers = preprocessing(submission_data, is_submission=True)[0], preprocessing(submission_data, is_submission=True)[1]
submission_df_prepared = full_pipeline.transform(submission_df)
predictions = model.predict(submission_df_prepared)
submission_df['Survived'] = predictions
submission_df['PassengerId'] = submission_passengers
submission = pd.DataFrame(submission_df[['PassengerId', 'Survived']])
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [67]:
# Save predictions to csv
submission.to_csv('submission1.csv', index=False)

In [68]:
# Save model to pickle file
pkl.dump(model, open('titanic_svc.pkl', 'wb'))